In [76]:
import pandas as pd
weather = pd.read_csv("../data/weather_data.csv")
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3253 entries, 0 to 3252
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NAME             3253 non-null   object 
 1   LATITUDE         3253 non-null   object 
 2   LONGITUDE        3253 non-null   float64
 3   ELEVATION        3253 non-null   float64
 4   DATE             3253 non-null   float64
 5   DAPR             3253 non-null   object 
 6   DAPR_ATTRIBUTES  4 non-null      float64
 7   MDPR             4 non-null      object 
 8   MDPR_ATTRIBUTES  4 non-null      float64
 9   PRCP             4 non-null      object 
 10  PRCP_ATTRIBUTES  3210 non-null   float64
 11  SNOW             3210 non-null   object 
 12  SNOW_ATTRIBUTES  3244 non-null   float64
 13  SNWD             3244 non-null   object 
 14  SNWD_ATTRIBUTES  3241 non-null   float64
 15  TMAX             3241 non-null   object 
 16  TMAX_ATTRIBUTES  3202 non-null   float64
 17  TMIN          

In [77]:
# Shift the columns, this removes the NAME data, but would be removed anyway
weather = weather.shift(-1, axis=1)

# Remove unneeded columns
cols_to_remove = ["NAME", "LATITUDE", "LONGITUDE", "ELEVATION", "DAPR", "DAPR_ATTRIBUTES", "MDPR", "MDPR_ATTRIBUTES", "PRCP_ATTRIBUTES", "SNOW_ATTRIBUTES", "SNWD_ATTRIBUTES", "TMAX_ATTRIBUTES", "TMIN_ATTRIBUTES", "TOBS_ATTRIBUTES", "WT01", "WT01_ATTRIBUTES", "WT11", "WT11_ATTRIBUTES"]

weather = weather.drop(columns=cols_to_remove)

In [78]:
# mm->in and C->F
def convert_to_freedom_units(row):
    mm_to_inches = lambda mm: mm / 25.4
    c_to_f = lambda c: (c * 9/5) + 32

    if row.name in ['PRCP', 'SNOW', 'SNWD']:
        return mm_to_inches(row)
    elif row.name in ['TMAX', 'TMIN', 'TOBS']:
        return c_to_f(row)
    else:
        return row

# Case date to a datetime object in pandas
weather["DATE"] = pd.to_datetime(weather["DATE"])
# Apply freedom units
weather = weather.apply(convert_to_freedom_units, axis=0)

/var/folders/rw/kqv4w2bj5hx2v1ln61zfm4pr0000gn/T/ipykernel_93256/1667900955.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather["DATE"] = pd.to_datetime(weather["DATE"])


In [79]:
# Fill all NaNs in specified columns by averaging the nearest non-null values before and after each missing value. Guarantees no NaNs remain in those columns.
def fill_all_missing_with_surrounding_avg(df, columns):

    df_filled = df.copy()

    for col in columns:
        series = df_filled[col]

        # Iterate until no NaNs remain
        while series.isna().any():
            nan_indices = series[series.isna()].index

            for idx in nan_indices:
                prev_idx = series.loc[:idx].last_valid_index()
                next_idx = series.loc[idx:].first_valid_index()

                prev_val = series[prev_idx] if prev_idx is not None else None
                next_val = series[next_idx] if next_idx is not None else None

                if prev_val is not None and next_val is not None:
                    series.at[idx] = (prev_val + next_val) / 2
                elif prev_val is not None:
                    series.at[idx] = prev_val
                elif next_val is not None:
                    series.at[idx] = next_val
                # If both are still None (entire column is NaN), you'd need fallback
                # But this loop ensures it keeps going until it's full

        df_filled[col] = series

    return df_filled

weather = fill_all_missing_with_surrounding_avg(weather, ['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS'])

In [80]:
weather.to_csv("../data/clean/weather_clean.csv", index=False)